In [1]:
import numpy as np
import matplotlib.pyplot as plt
import splat.simulate as spsim
import wisps.simulations as wispsim
import emcee
from astropy.coordinates import SkyCoord
import astropy.units as u
%matplotlib inline

In [2]:
def lnprob(d, dmax, dmin, coord):
    """
    probability density in all directions
    """
    if (d>dmax) or (d<dmin) :
        return -np.inf
    else:
        new_coord=SkyCoord(ra=coord.ra, dec=coord.dec, distance=d*u.kpc)
        r=(new_coord.cartesian.x**2+new_coord.cartesian.y**2)**0.5
        return np.log((d**3)*spsim.galactic_density_juric(r, new_coord.cartesian.z, unit=u.kpc, center='sun').value)

In [3]:
class Pointing(object):
    
    def __init__(self, **kwargs):
        ## a random pointing
        self.direction=kwargs.get('direc', None)
        self.ensemble_samplers=[]
    
    def sample(self, dmin, dmax, nsample):
        ##run MCMC 
        ndim=1
        nwalkers = 250
        p0 = np.random.rand(ndim * nwalkers).reshape((nwalkers, ndim))
        sampler =emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(dmax, dmin, self.direction))
        sampler.run_mcmc(p0, nsample)
        self.ensemble_samplers.append(sampler)
        
    @property
    def samples(self):
        return np.array([x.flatchain for x in self.ensemble_samplers])
    @property
    def lnprobability(self):
        return  np.array([np.concatenate(x.lnprobability) for x in self.ensemble_samplers])
        

In [4]:
pnts=[Pointing(direc=x) for x in wispsim.OBSERVED_POINTINGS]

In [5]:
#%%capture 
#list(map(lambda x: x.sample(0.01, 2.0, 10), pnts))

In [8]:
p=Pointing(direc=wispsim.OBSERVED_POINTINGS[-1])

In [9]:
dlimits, spgrid, vols, vcs=wispsim.compute_effective_volumes()

In [10]:
%%capture
list(map(lambda x: [x.sample(dlim[0], dlim[1], 10) for dlim in dlimits], pnts))#[(p, dlim)for dlim, p in zip(dlimits, pnts)]

In [11]:
import wisps
import pickle
with open(wisps.OUTPUT_FILES+'/eff_distances.pkl', 'wb') as file:
    pickle.dump(pnts, file)

In [4]:
import emcee
emcee.__version__

'2.2.1'

In [15]:
pnts[0].samples.shape

(18, 2500, 1)

In [1]:
import wisps.simulations as wispsim

#%timeit -o wispsim.compute_effective_distances(recompute=True, nsamples=5)

In [2]:
import subprocess
import matplotlib.pyplot as plt
import numpy as np

In [3]:
dlimits, spgrid, vols, vcs=wispsim.compute_effective_volumes()
pnts=[wispsim.Pointing(direc=x) for x in wispsim.OBSERVED_POINTINGS]

In [4]:
x=tuple([np.diff(dlimits)[:,0], np.array(pnts)])

In [5]:
from itertools import product

In [6]:
x=np.diff(dlimits)[:,0]
y=np.array(pnts)
c = list(product(x, y))

In [7]:
from multiprocessing import Pool
from functools import partial
from numba import jit


def recompute_stuff(comb):
    d,sampler= comb
    sampler.sample(0, d, 10)
    return sampler.samples

from concurrent.futures import ThreadPoolExecutor

executor = ThreadPoolExecutor(max_workers=2)
a = executor.map(recompute_stuff, c)

In [16]:
a.result()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/envs/myenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-7de6d01dcb2f>", line 1, in <module>
    a.result()
AttributeError: 'generator' object has no attribute 'result'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/myenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2018, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/myenv/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/anaconda3/envs/

AttributeError: 'generator' object has no attribute 'result'

In [2]:
import multiprocessing

multiprocessing.cpu_count()

4

In [1]:
import numpy as np

In [2]:
arr=np.load('/Users/caganze/desktop/data0.npy')

AttributeError: Can't get attribute '_function_wrapper' on <module 'emcee.ensemble' from '/anaconda3/envs/myenv/lib/python3.7/site-packages/emcee-3.0rc2-py3.7.egg/emcee/ensemble.py'>

In [ ]:
import matplotlib.pyplot as plt
plt.hist(np.concatenate(arr))

In [8]:
%prun run 'run thescript.py'

         3 function calls in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)